<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/capitaliza%C3%A7%C3%A3o%20one%20mar%2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install pyxlsb

In [3]:
capit="Copy of unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3_2 - TESTE.xlsb"
folha="listagem upfront"
folha_pagar="Valor a pagar"

In [4]:
#carregar ficheiro dos alarmes

from google.colab import files
uploaded = files.upload()

Saving Copy of unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3_2 - TESTE.xlsb to Copy of unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3_2 - TESTE.xlsb


In [5]:
#ler folha auto

folha_capit=pd.read_excel(capit, sheet_name=folha)

In [6]:
#ler folha valor a pagar

folha_pagar=pd.read_excel(capit, sheet_name=folha_pagar)

In [10]:
#definir lista com o nome das colunas

colunas_pagar=["username", "chefe de equipa", "função", "parceiro", "Activações", "Correção autos anteriores", "Correções autos anteriores_histórico",\
"Correções autos anteriores a aplicar", "Total ativações", "Penalidades N", "Penalidades_histórico", "Penalidade total a aplicar", \
"Charge back N", "Charge back_histórico", "Charge back total a aplicar", "Ativação Negativa", "Ativação Negativa Histórico", "Ativação Negativa total a aplicar", \
"Melhoria Contínua", "Valor", "VALOR COM FATOR", "Correções a transitar", "Penalidades a transitar", "Charge back a transitar", "Ativações negativas a transitar", \
"Status CAV N-1", "Status CAV N", "Imputar ao CC do OBV_correções", "Imputar ao CC do OBV_penalidades", "Imputar ao CC do OBV_charge back", \
"Imputar ao CC do OBV_ativ neg", "Correções a transitar para o mês seguinte", "Penalidade a transitar para o mês seguinte", \
"Charge back a transitar para o mês seguinte", "Ativações negativas a transitar para o mês seguinte", "rede", "Auto", "CAV", "Valores_transitar", \
"Comentário", "activação negativa transitar", "comentário activações negativas", "ChargeBack transitar", "comentário chargeback", \
"Correcções transitar", "comentário Correcções", "Penalidades transitar", "comentário Penalidades"]
print(len(colunas_pagar))

48


In [11]:
#definir uma nova bd
folhas_pagar2=folha_pagar.iloc[:, :len(colunas_pagar)].dropna().iloc[1:, :]
folhas_pagar2.columns=colunas_pagar
#print(folhas_pagar2.shape)
#print(folhas_pagar2.columns)
folhas_pagar2.head()


,username,chefe de equipa,função,parceiro,Activações,Correção autos anteriores,Correções autos anteriores_histórico,Correções autos anteriores a aplicar,Total ativações,Penalidades N,...,Valores_transitar,Comentário,activação negativa transitar,comentário activações negativas,ChargeBack transitar,comentário chargeback,Correcções transitar,comentário Correcções,Penalidades transitar,comentário Penalidades
6,U80086295,U80037039,Operador,Heading,990.6,4.852174,0,4.852174,995.452174,0,...,0,ok,0,Sem NA,0,Sem CB,0,Sem Correcção,0,Sem Correcção
7,U80047659,U80037039,Operador,Heading,774.6,0,0,0,774.6,-32,...,0,ok,0,Sem NA,0,Sem CB,0,Sem Correcção,0,Sem Correcção
8,amargrodrigues,U80037039,Operador,Randstad,582.6,0,0,0,582.6,-32,...,0,ok,0,Sem NA,0,Sem CB,0,Sem Correcção,0,Sem Correcção
9,jfcsilva,U80037039,Operador,Heading,111.75,0,0,0,111.75,-6,...,0,ok,0,Sem NA,0,Sem CB,0,Sem Correcção,0,Sem Correcção
10,PJBARROSO,TLCOIMBRA,Operador,Heading,939.4,-5.12,0,-5.12,934.28,-32,...,0,ok,0,Sem NA,0,Sem CB,0,Sem Correcção,0,Sem Correcção


In [23]:
dici_rubrica_anular={"correcoes": ["Correções a transitar", "Correção autos anteriores"], \
"activacoes_neg": ["Ativações negativas a transitar", "Ativação Negativa"], \
"cb": ["Charge back a transitar", "Charge back N"], \
"penalidade": ["Penalidade a transitar para o mês seguinte", "Penalidades N"]}

In [27]:
#identificar linhas com correcções a anular
def correcoes_anular(df, colacti, coltransseguinte, colcorr, status1, status2):
  #identificar pessoas com correcções maiores que zero e com correcções a transitar para o mês seguinte
  df2=df.loc[(df[colcorr]==df[coltransseguinte]) & (df[colacti]==0) & (df[colcorr]!=0), :]
  #identificar pessoas com correcções de auto anteriores e com status inactivo
  df3=df.loc[(df[colcorr]!=0) & (df[status1]=="INACTIVO") & (df[status2]=="INACTIVO"),:]
  df_anular=pd.concat([df2, df3], axis=0)

  return df_anular["username"].tolist()


In [41]:
dici_rubrica_corrigir={"correccao": ["Correção autos anteriores", "Correções autos anteriores a aplicar"], \
"activação neg": ["Ativação Negativa", "Ativação Negativa total a aplicar"], \
"chargeback": ["Charge back N", "Charge back total a aplicar"], \
"penalidade": ["Penalidades N", "Penalidade total a aplicar"]}

In [86]:
#identificar linhas com eventos a transitar

def correcoes (df, colacti, colcorr, coltrans):
  #df2 tem pessoas com activações, com correcções e com correcções a serem aplicadas
  df2=df.loc[(df[colacti]>0) & (df[colcorr]!=0) & (df[coltrans]!=0),:]
  #df2 separa as pessoas em que se aplica a correcção total e aquelas que é preciso ratear
  df2["status"]=df2[colcorr].where(abs(df2[colcorr])>abs(df2[coltrans]), other="não ratear")
  df3=df2.loc[df2["status"]!="não ratear", :]

  lista=df3[["username", "função", colcorr, coltrans]].apply(lambda i: (i[0], i[1], i[2], i[3]), axis=1).tolist()

  return lista

  #return df3[["username", "função", colcorr, coltrans]]
correcoes(folhas_pagar2, "Activações", "Correção autos anteriores", "Correções autos anteriores a aplicar")
#df_ajustes=correcoes(folhas_pagar2, "Activações", "Correção autos anteriores", "Correções autos anteriores a aplicar")



<ipython-input-86-1c3d38a3cafe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["status"]=df2[colcorr].where(abs(df2[colcorr])>abs(df2[coltrans]), other="não ratear")


[('sbmarante', 'Operador', -5, -2), ('pspaulino', 'Operador', -5.35, -1)]

In [ ]:

#folha_capit2=folha_capit.copy()

#for user, func, val_or, val_ap in lista:
  #if func=="Operador":
    #folha_capit2["correcção operador"]=np.where((folha_capit["Username operador"].isna()==False) & (folha_capit["Username operador"]==user) & \
    #(folha_capit["Tipo"].isin(["Correcções marcações", "Correcções autos anteriores"])), folha_capit["Total € operador"]*val_ap/val_or, \
    #folha_capit["Total € operador"])

  #print(folha_capit2.loc[(folha_capit2["Username operador"]==user) & \
  #(folha_capit["Tipo"].isin(["Correcções marcações", "Correcções autos anteriores"])), "correcção operador"])


10187   -0.4
10189   -0.4
10238   -0.4
10310   -0.4
10314   -0.4
Name: correcção operador, dtype: float64
9076    -0.252336
10270   -0.186916
10271   -0.186916
10272   -0.186916
10329   -0.186916
Name: correcção operador, dtype: float64


In [ ]:
#aplicável aos valores a transitar - define os novos valores e os índices onde os mesmos vão ser aplicados

dici_op=[]
indice=[]

for user, func, val_or, val_ap in lista:
  if func=="Operador":
    dici_op +=folha_capit.loc[(folha_capit["Username operador"]==user) & \
    (folha_capit["Tipo"].isin(["Correcções marcações", "Correcções autos anteriores"])), "Total € operador"].apply(lambda i: \
    i*val_ap/val_or).tolist()

    indice +=folha_capit.loc[(folha_capit["Username operador"]==user) & \
    (folha_capit["Tipo"].isin(["Correcções marcações", "Correcções autos anteriores"])), ["Total € operador"]].index.tolist()

print(dici_op)

print(indice)





[-0.4, -0.4, -0.4, -0.4, -0.4, -0.25233644859813076, -0.1869158878504673, -0.1869158878504673, -0.1869158878504673, -0.1869158878504673]
[10187, 10189, 10238, 10310, 10314, 9076, 10270, 10271, 10272, 10329]


In [ ]:
#substituição dos valores antigos pelos novos valores

for i, j in zip(indice, dici_op):
  print(j)
  folha_capit2.loc[i]=j

  folha_capit2.loc[(folha_capit2["Username operador"]=="pspaulino") & \
  (folha_capit2["Tipo"].isin(["Correcções marcações", "Correcções autos anteriores"])),"Total € operador"]


-0.4
-0.4
-0.4
-0.4
-0.4
-0.25233644859813076
-0.1869158878504673
-0.1869158878504673
-0.1869158878504673
-0.1869158878504673


In [124]:
def valor_indice (df, rubrica, user, func, val_ap, val_or):
  dici_op=[]
  indice=[]
  dici_rubrica={"correccao": ["Correcções marcações", "Correcções autos anteriores"], "activação neg": ["Activações negativas"], \
  "penalidade": ["Penalidades", "Penalidades qualidade comercial"], "chargeback": ["Chargeback", "Chargeback SDD"]}
  if func=="Operador":
    dici_op +=df.loc[(df["Username operador"]==user) & \
    (df["Tipo"].isin(dici_rubrica[rubrica])), "Total € operador"].apply(lambda i: \
    i*val_ap/val_or).tolist()

    indice +=df.loc[(df["Username operador"]==user) & \
    (df["Tipo"].isin(dici_rubrica[rubrica])), ["Total € operador"]].index.tolist()

  elif func=="Supervisor":
    dici_op +=df.loc[(df["Username supervisor"]==user) & \
    (df["Tipo"].isin(dici_rubrica[rubrica])), "Total € supervisor"].apply(lambda i: \
    i*val_ap/val_or).tolist()

    indice +=df.loc[(df["Username supervisor"]==user) & \
    (df["Tipo"].isin(dici_rubrica[rubrica])), ["Total € supervisor"]].index.tolist()

  elif func=="Supervisor TC":
    dici_op +=df.loc[(df["Username supervisor training center"]==user) & \
    (df["Tipo"].isin(dici_rubrica[rubrica])), "Total € supervisor TC"].apply(lambda i: \
    i*val_ap/val_or).tolist()

    indice +=df.loc[(df["Username supervisor training center"]==user) & \
    (df["Tipo"].isin(dici_rubrica[rubrica])), ["Total € supervisor TC"]].index.tolist()

  else:
    dici_op +=df.loc[(df["Username coordenador"]==user) & \
    (df["Tipo"].isin(dici_rubrica[rubrica])), "Total € coordenador"].apply(lambda i: \
    i*val_ap/val_or).tolist()

    indice +=df.loc[(df["Username coordenador"]==user) & \
    (df["Tipo"].isin(dici_rubrica[rubrica])), ["Total € coordenador"]].index.tolist()

  return dici_op, indice


In [125]:
folha_capit2=folha_capit.copy()
for rub in dici_rubrica_corrigir.keys():
  rubrica=rub
  for i, j in dici_rubrica_corrigir.items():
    #print(i, j)
    if len(correcoes(folhas_pagar2, "Activações", j[0], j[1]))>0:
      for cont in correcoes(folhas_pagar2, "Activações", j[0], j[1]):
        user, func, val1, val2=cont
        dicionario, indice=valor_indice(folha_capit, rubrica, user, func, val1, val2)
        for i, j in zip(indice, dicionario):
          folha_capit2.loc[i]=j


<ipython-input-86-1c3d38a3cafe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["status"]=df2[colcorr].where(abs(df2[colcorr])>abs(df2[coltrans]), other="não ratear")
<ipython-input-86-1c3d38a3cafe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["status"]=df2[colcorr].where(abs(df2[colcorr])>abs(df2[coltrans]), other="não ratear")
<ipython-input-86-1c3d38a3cafe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [121]:

for i, j in dici_rubrica_anular.items():
  lista_anular=correcoes_anular(folhas_pagar2, "Activações", j[0], j[1], \
  "Status CAV N-1", "Status CAV N")
  for x in lista_anular:
    funcao=folhas_pagar2.loc[folhas_pagar2["username"]==x, "função"].tolist()[0]
    if funcao=="Operador":
      folha_capit2.loc[(folha_capit2["Username operador"]==x) & (folha_capit2["Tipo"].isin(j)), "Total € operador"]=0
    elif funcao=="Supervisor":
      folha_capit2.loc[(folha_capit2["Username supervisor"]==x) & (folha_capit2["Tipo"].isin(j)), "Total € supervisor"]=0
    elif funcao=="Supervisor TC":
      folha_capit2.loc[(folha_capit2["Username supervisor training center"]==x) & (folha_capit2["Tipo"].isin(j)), "Total € supervisor TC"]=0
    else:
      folha_capit2.loc[(folha_capit2["Username coordenador"]==x) & (folha_capit2["Tipo"].isin(j)), "Total € coordenador"]=0


In [122]:
folha_capit2.to_excel("output.xlsx", index=False)

In [123]:
files.download('output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Sample DataFrame - sample
data = {'A': [1, 2, 3, 4, 5],
        'B': [10, 20, 30, 40, 50]}
df = pd.DataFrame(data)

# List of replacement values
replacement_list = [{'A': 100, 'B': 200}, {'A': 300, 'B': 400}]

# Specify the row indices you want to replace
replace_indices = [1, 3]

# Replace the rows with the values from the list
for idx, new_values in zip(replace_indices, replacement_list):
    df.loc[idx] = new_values

print(df)